<a href="https://colab.research.google.com/github/rebirthmagex/ABSA/blob/main/board.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from contextlib import suppress
from pprint import pprint

import math
from enum import Enum
import copy
from copy import deepcopy
from collections import deque

import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
!pip install import-ipynb
import import_ipynb

In [ ]:
from point import Point

In [ ]:
class Board:
    # Tipos Enumerados
    class Player(Enum):
        TIGER = 1
        GOAT = 2

    class MoveType(Enum):
        PLACE = 1  # Colocar cabra
        MOVE = 2   # Mover cabra ou tigre
        CAPTURE = 3  # Capturar cabra

    # Constantes do jogo

    EMPTY = "."
    TIGER = "T"
    GOAT = "C"

    def __init__(self):
        self.point = Point() # Cria uma instância de `Point` dentro de `Board`
        self.game_board = self._initialize_board()
        self.new_board = deepcopy(self.game_board)
        self.tiger_positions = ['A1', 'E1', 'A5', 'E5']
        self.board_size = 5
        self.goats_to_be_placed = 20
        self.dead_goats = 0
        self.goats_to_win = 5
        self.turn = self.Player.GOAT  # Cabras começam o jogo

    def _initialize_board(self):
        """
        Inicializa o tabuleiro do jogo com a posição inicial dos tigres.
        """
        # Aqui, referenciamos TIGER e EMPTY diretamente
        board = [
            [self.TIGER, self.EMPTY, self.EMPTY, self.EMPTY, self.TIGER],
            [self.EMPTY, self.EMPTY, self.EMPTY, self.EMPTY, self.EMPTY],
            [self.EMPTY, self.EMPTY, self.EMPTY, self.EMPTY, self.EMPTY],
            [self.EMPTY, self.EMPTY, self.EMPTY, self.EMPTY, self.EMPTY],
            [self.TIGER, self.EMPTY, self.EMPTY, self.EMPTY, self.TIGER]
        ]
        return board

    def print_board(self, game_board=None):
        """
        Imprime o tabuleiro escolhido. Se nenhum tabuleiro for passado,
        imprime o tabuleiro original (`self.board`).
        """
        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

        for row in game_board:
            print(" ".join(row))
        print()

    def show(self, game_board=None):

        print(f"\nTurno: {'Cabras' if self.turn == self.Player.GOAT else 'Tigres'}")

        if game_board is None:
            self.print_board(self.game_board)
        else:
            self.print_board(game_board)

        print(f"Cabras restantes para colocar: {self.goats_to_be_placed}")
        print(f"Cabras mortas: {self.dead_goats}\n")

    def extract_value_from_position(self, position, game_board=None):
      """
      Extrai o valor que está em um position do tabuleiro.
      """
      if game_board is None:
        game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

      x, y = self.point.COORD_TO_CARTESIAN_PLANE[position]  # Converte para coordenadas no plano cartesiano
      value = game_board[x][y]  # Acessa o valor no tabuleiro
      return value

    ################################### Funções de movimentação ###################################

    def place_goat(self, position, game_board=None):
        """
        Coloca uma cabra em uma cópia do tabuleiro e retorna essa nova cópia,
        sem modificar o tabuleiro original.
        """

        x, y = self.point.COORD_TO_CARTESIAN_PLANE.get(position)


        if game_board is None:
          game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

          if game_board[x][y] == self.EMPTY and self.goats_to_be_placed > 0:
              game_board[x][y] = self.GOAT
              self.goats_to_be_placed -= 1

              self.game_board = game_board

        else:

          self.new_board = copy.deepcopy(game_board)

          if self.new_board[x][y] == self.EMPTY and self.goats_to_be_placed > 0:
              self.new_board[x][y] = self.GOAT
              self.goats_to_be_placed -= 1

          return self.new_board

    def is_valid_position(self, coord):
        return coord in self.point.COORD_TO_INDEX

    def is_movable(self, from_coord, to_coord, game_board=None):
        """
        Verifica se uma peça pode se mover de um ponto específico para outro.
        Exemplo: 'A1' para 'B2'
        """

        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

        # Verifica se ambos os pontos são válidos
        if not (self.is_valid_position(from_coord) and self.is_valid_position(to_coord)):
            return False

        return (
            # A conexão deve existir
            to_coord in self.point._move_connections[from_coord] and
            # O ponto de destino deve estar vazio
            self.extract_value_from_position(to_coord) == self.EMPTY
        )

    def can_capture(self, from_coord, to_coord, game_board=None):
            """
            Verifica se um tigre pode capturar uma cabra movendo-se de uma coordenada para outra em uma cópia do tabuleiro,
            sem modificar o tabuleiro original.
            """
            if game_board is None:
                game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

            if not (self.is_valid_position(from_coord) and self.is_valid_position(to_coord)):
                return False

            if to_coord in self.point._capture_connections[from_coord]:
                fx, fy = self.point.COORD_TO_CARTESIAN_PLANE.get(from_coord)
                tx, ty = self.point.COORD_TO_CARTESIAN_PLANE.get(to_coord)
                mx, my = (fx + tx) // 2, (fy + ty) // 2
                return game_board[tx][ty] == self.EMPTY and self.game_board[mx][my] == self.GOAT
            return False

    def move_piece(self, from_coord, to_coord, game_board=None):
        """
        Move uma peça de uma coordenada para outra em uma cópia do tabuleiro,
        sem modificar o tabuleiro original, e retorna essa nova cópia.
        """
        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

            if self.is_movable(from_coord, to_coord, game_board):
                fx, fy = self.point.COORD_TO_CARTESIAN_PLANE.get(from_coord)
                tx, ty = self.point.COORD_TO_CARTESIAN_PLANE.get(to_coord)
                game_board[tx][ty] = game_board[fx][fy]
                game_board[fx][fy] = self.EMPTY

            if self.can_capture(from_coord, to_coord, game_board):
                fx, fy = self.point.COORD_TO_CARTESIAN_PLANE.get(from_coord)
                tx, ty = self.point.COORD_TO_CARTESIAN_PLANE.get(to_coord)
                mx, my = (fx + tx) // 2, (fy + ty) // 2
                game_board[tx][ty] = self.TIGER
                game_board[fx][fy] = self.EMPTY
                game_board[mx][my] = self.EMPTY
                self.dead_goats += 1

            self.game_board = game_board

        else:

          # Cria uma cópia do tabuleiro para modificar
          self.new_board = copy.deepcopy(game_board)

          if self.is_movable(from_coord, to_coord, self.new_board):
              fx, fy = self.point.COORD_TO_CARTESIAN_PLANE.get(from_coord)
              tx, ty = self.point.COORD_TO_CARTESIAN_PLANE.get(to_coord)
              self.new_board[tx][ty] = self.new_board[fx][fy]
              self.new_board[fx][fy] = self.EMPTY
              return self.new_board

          if self.can_capture(from_coord, to_coord, self.new_board):
              fx, fy = self.point.COORD_TO_CARTESIAN_PLANE.get(from_coord)
              tx, ty = self.point.COORD_TO_CARTESIAN_PLANE.get(to_coord)
              mx, my = (fx + tx) // 2, (fy + ty) // 2
              self.new_board[tx][ty] = self.TIGER
              self.new_board[fx][fy] = self.EMPTY
              self.new_board[mx][my] = self.EMPTY
              self.dead_goats += 1
              return self.new_board

          return self.new_board  # Retorna o tabuleiro sem modificações se o movimento não for possível

    ################################### Funções de Geração de Movimentos ###################################

    def _get_tiger_positions(self, game_board=None):
        """
        Encontra as posições dos tigres no tabuleiro e as salva na lista `tiger_positions`.
        Trabalha em uma cópia do tabuleiro, se fornecida.
        """

        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

        tiger_positions = []
        tiger_positions = [position for position in self.point.COORD_TO_CARTESIAN_PLANE.keys() if self.extract_value_from_position(position) == self.TIGER]

        return tiger_positions

    def _get_goat_positions(self, game_board=None):
        """
        Encontra as posições dos tigres no tabuleiro e as salva na lista `tiger_positions`.
        Trabalha em uma cópia do tabuleiro, se fornecida.
        """

        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

        goat_positions = []
        goat_positions = [position for position in self.point.COORD_TO_CARTESIAN_PLANE.keys() if self.extract_value_from_position(position) == self.GOAT]

        return goat_positions

    def _get_empty_positions(self, game_board=None):
        """
        Encontra as posições dos tigres no tabuleiro e as salva na lista `tiger_positions`.
        Trabalha em uma cópia do tabuleiro, se fornecida.
        """

        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

        empty_positions = []
        empty_positions = [position for position in self.point.COORD_TO_CARTESIAN_PLANE.keys() if self.extract_value_from_position(position) == self.EMPTY]

        return empty_positions

    def get_tiger_moves(self, game_board=None):
        """
        Gera uma lista de todos os movimentos possíveis dos tigres para o turno atual do tabuleiro.
        """

        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

        move_list = []

        for from_coord in self._get_tiger_positions(game_board):
            for to_coord in self.point.get_connections(from_coord):

                # Verifica se o movimento é possível
                if self.is_movable(from_coord, to_coord, game_board):
                    move_list.append({'from': from_coord, 'to': to_coord, 'type': self.MoveType.MOVE})

                # Verifica se a captura é possível
                elif self.can_capture(from_coord, to_coord, game_board):
                    move_list.append({'from': from_coord, 'to': to_coord, 'type': self.MoveType.CAPTURE})

        return move_list

    def get_goat_moves(self, game_board=None):
        """
        Gera uma lista de todos os movimentos possíveis das cabras para o turno atual do tabuleiro.
        """

        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

        move_list = []

        # Fase de colocação das cabras
        if self.goats_to_be_placed > 0:
          for to_coord in self._get_empty_positions(game_board):
              move_list.append({'from': None, 'to': to_coord, 'type': self.MoveType.PLACE})

        else: # Fase de movimentação das cabras

            for from_coord in self._get_goat_positions(game_board):
                for to_coord in self.point._move_connections[from_coord]:
                    if self.is_movable(from_coord, to_coord, game_board):
                        move_list.append({'from': from_coord, 'to': to_coord, 'type': self.MoveType.MOVE})

        return move_list

    def generate_move_list(self, game_board=None):
        """
        Gera uma lista de todos os movimentos possíveis para o turno atual do tabuleiro.
        """
        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

        move_list = []

        # Se for o turno das cabras
        if self.turn == self.Player.GOAT:

          move_list = self.get_goat_moves(game_board)

        # Se for o turno dos tigres
        else:

          move_list = self.get_tiger_moves(game_board)

        return move_list

    def make_move(self, move, game_board=None):
        """
        Realiza o movimento dado no tabuleiro usando o formato de dicionário gerado por generate_move_list().
        """

        from_coord = move['from']
        to_coord = move['to']
        move_type = move['type']

        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

            # Colocação da cabra
            if move_type == self.MoveType.PLACE:
              self.place_goat(to_coord)
              self.turn = self.Player.TIGER

            # Movimento de cabra ou tigre
            elif move_type == self.MoveType.MOVE:
                if self.turn == self.Player.GOAT:
                  self.move_piece(from_coord, to_coord)
                  self.turn = self.Player.TIGER

                else:
                  self.move_piece(from_coord, to_coord)
                  self.tiger_positions = self._get_tiger_positions()
                  self.turn = self.Player.GOAT

            # Captura por um tigre
            elif move_type == self.MoveType.CAPTURE:
              self.move_piece(from_coord, to_coord)
              self.tiger_positions = self._get_tiger_positions()
              self.turn = self.Player.GOAT

            #atualiza o tabuleiro
            self.game_board = game_board

        else:
          self.new_board = copy.deepcopy(game_board)

          # Colocação da cabra
          if move_type == self.MoveType.PLACE:
            self.new_board = self.place_goat(to_coord, self.new_board)
            self.turn = self.Player.TIGER

          # Movimento de cabra ou tigre
          elif move_type == self.MoveType.MOVE:
              if self.turn == self.Player.GOAT:
                self.new_board = self.move_piece(from_coord, to_coord, self.new_board)
                self.tiger_positions = self.turn = self.Player.TIGER

              else:
                self.new_board = self.move_piece(from_coord, to_coord, self.new_board)
                self.tiger_positions = self._get_tiger_positions()
                self.turn = self.Player.GOAT

          # Captura por um tigre
          elif move_type == self.MoveType.CAPTURE:
            self.new_board = self.move_piece(from_coord, to_coord, self.new_board)
            self.tiger_positions = self._get_tiger_positions()
            self.turn = self.Player.GOAT

          return self.new_board

    def set_state(self, coord, state, game_board=None):

      if game_board is None:
        game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

      x, y = self.point.COORD_TO_CARTESIAN_PLANE[coord]
      game_board[x][y] = state


    def revert_move(self, move, game_board=None):
        """
        Reverte o movimento dado no tabuleiro.

        Parameters:
        - move: Um dicionário que contém 'from', 'to', e 'type' do movimento.
        - board: O tabuleiro em que o movimento deve ser revertido. Se não for fornecido, o tabuleiro principal é usado.

        """

        from_coord = move['from']
        to_coord = move['to']
        move_type = move['type']

        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

        # Reverte a colocação de uma cabra
        if move_type == self.MoveType.PLACE:
            # Restaura o espaço vazio onde a cabra foi colocada
            self.set_state(to_coord, self.EMPTY, game_board)
            self.turn = self.Player.GOAT
            self.goats_to_be_placed += 1

        # Reverte o movimento de uma peça (cabra ou tigre)
        elif move_type == self.MoveType.MOVE:
            # Restaura as posições das peças após o movimento
            x_from, y_from = self.point.COORD_TO_CARTESIAN_PLANE[from_coord]
            x_to, y_to = self.point.COORD_TO_CARTESIAN_PLANE[to_coord]

            if self.turn == self.Player.GOAT:
                self.set_state(from_coord, self.TIGER, game_board)
                self.set_state(to_coord, self.EMPTY, game_board)
                self.turn = self.Player.TIGER
            else:
                self.set_state(from_coord, self.GOAT, game_board)
                self.set_state(to_coord, self.EMPTY, game_board)
                self.turn = self.Player.GOAT

        # Reverte a captura feita por um tigre
        elif move_type == self.MoveType.CAPTURE:
            # Restaura a peça capturada e move o tigre de volta
            x_from, y_from = self.point.COORD_TO_CARTESIAN_PLANE[from_coord]
            x_to, y_to = self.point.COORD_TO_CARTESIAN_PLANE[to_coord]
            mid_x, mid_y = (x_from + x_to) // 2, (y_from + y_to) // 2

            game_board[x_from][y_from] = self.TIGER
            game_board[mid_x][mid_y] = self.GOAT
            game_board[x_to][y_to] = self.EMPTY
            self.turn = self.Player.TIGER
            self.dead_goats -= 1

        # Atualiza o estado das posições dos tigres
        self.tiger_positions = self._get_tiger_positions(game_board)

        # Atualiza o tabuleiro
        self.game_board = game_board


    ################################### Funções de Parada (Vitória) ###################################

    def count_movable_tigers(self, game_board=None):
        """
        Conta o número de tigres que podem se mover ou capturar.

        Parameters:
        - board: O tabuleiro analisado.

        Returns:
        - O número de tigres que podem se mover ou capturar.
        """

        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

        movable_tigers_count = 0

        for from_coord in self._get_tiger_positions(game_board):
            if any(self.is_movable(from_coord, to_coord, game_board) for to_coord in self.point.get_connections(from_coord)):
                movable_tigers_count += 1

        return movable_tigers_count

    def _all_tigers_trapped(self, game_board=None):
            """
            Retorna True se não houver movimentos válidos restantes para os tigres
            em uma cópia do tabuleiro fornecido, sem modificar o tabuleiro original.
            """
            if game_board is None:
                game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

            return False if self.count_movable_tigers(game_board) >=1 else True

    @property
    def winner(self):
        """
        Retorna o vencedor se o jogo terminou, caso contrário retorna None.
        Verifica o estado atual do jogo sem modificar o tabuleiro original.
        """

        if self.dead_goats == self.goats_to_win:
            return self.Player.TIGER

        if self._all_tigers_trapped():
            return self.Player.GOAT

        return None


    ################ Funções de Avaliação ################
    def _is_closed(self, position, game_board=None):
        """
        Retorna True se a posição estiver fechada; caso contrário, retorna False.
        --------------------------------------------------
        Fechado significa que a posição está vazia e cercada
        por todas as cabras vizinhas. Além disso, nenhum tigre
        pode acessar a posição vazia capturando.
        """

        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

        # Verifica se todos os valores nas posições conectadas a position são cabras
        all_goats = all(
            self.extract_value_from_position(value, game_board) == self.GOAT
            for value in self.point.get_connections(position)
        )

        return all_goats

    def no_of_closed_spaces(self, game_board=None):
        """
        Return the number of closed spaces in the board.
        """

        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

        return len([True for i in self._get_empty_positions(game_board) if self._is_closed(i, game_board)])

    def manhattan_distance(self, pos1, pos2):
        """
        Calcula a distância de Manhattan entre duas posições.
        """
        x1, y1 = Point.COORD_TO_CARTESIAN_PLANE[pos1]
        x2, y2 = Point.COORD_TO_CARTESIAN_PLANE[pos2]
        return abs(x1 - x2) + abs(y1 - y2)

    def count_moves_to_adjacent_goat_manhattan(self, tiger_position, game_board=None):
        """
        Conta o número mínimo de jogadas necessárias para que o tigre na posição
        `tiger_position` fique adjacente a uma cabra, usando a distância de Manhattan.
        """
        if game_board is None:
            game_board = self.game_board

        min_distance = math.inf
        for position in Point.COORD_TO_CARTESIAN_PLANE.keys():
            if self.extract_value_from_position(position, game_board) == self.GOAT:
                distance = self.manhattan_distance(tiger_position, position) - 1  # -1 para adjacência
                min_distance = min(min_distance, distance)

        return min_distance

    def count_moves_to_adjacent_goat(self, tiger_position, game_board=None):
        """
        Conta o número mínimo de jogadas necessárias para que o tigre na posição
        `tiger_position` fique adjacente a uma cabra.
        """
        if game_board is None:
            game_board = self.game_board

        queue = deque([(tiger_position, 0)])  # (posição atual, número de movimentos)
        visited = set()

        while queue:
            current_position, distance = queue.popleft()
            visited.add(current_position)

            # Verifica se há uma cabra adjacente
            for neighbor in Point.get_connections(current_position):
                if neighbor in visited:
                    continue

                # Se há uma cabra adjacente, retorne o número de movimentos necessários
                if self.extract_value_from_position(neighbor, game_board) == self.GOAT:
                    return distance + 1

                # Se o espaço está vazio, adicione à fila para exploração
                if self.extract_value_from_position(neighbor, game_board) == self.EMPTY:
                    queue.append((neighbor, distance + 1))

        # Se não há nenhuma cabra acessível, retorne um número grande
        return math.inf

    def get_amount_of_tiger_moves(self, game_board=None):
        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

        return len(self.get_tiger_moves(game_board))

    def get_amount_of_goat_moves(self, game_board=None):
        if game_board is None:
            game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado

        return len(self.get_goat_moves(game_board))

    def tiger_potential_captures(self, game_board=None):

      if game_board is None:
        game_board = self.game_board  # Usa o tabuleiro original se nenhum outro for especificado


      tiger_moves = self.get_tiger_moves(self.game_board)
      return sum(1 for move in tiger_moves if move['type'] == self.MoveType.CAPTURE)

    def occupy_important_lines(self, game_board=None):
        if game_board is None:
            game_board = self.game_board

        important_positions = ['C1', 'C2', 'C3', 'C4', 'C5', 'B2', 'B3', 'B4', 'D2', 'D3', 'D4']
        tiger_positions = self._get_tiger_positions(game_board)
        return sum(1 for pos in tiger_positions if pos in important_positions)

    def avoid_trapping(self, game_board=None):
        if game_board is None:
            game_board = self.game_board

        trapped_tigers = len([tiger for tiger in self._get_tiger_positions(game_board)
                              if not any(self.is_movable(tiger, move, game_board)
                                        for move in self.point.get_connections(tiger))])
        return trapped_tigers

    def group_attack(self, game_board=None):
        if game_board is None:
            game_board = self.game_board

        tiger_positions = self._get_tiger_positions(game_board)
        return -sum(self.manhattan_distance(t1, t2) for i, t1 in enumerate(tiger_positions) for t2 in tiger_positions[i+1:])

    def minimize_captures(self, game_board=None):
        if game_board is None:
            game_board = self.game_board

        tiger_moves = self.get_tiger_moves(game_board)
        return sum(1 for move in tiger_moves if move['type'] == self.MoveType.CAPTURE)

    def maximize_protection(self, game_board=None):
        if game_board is None:
            game_board = self.game_board

        goat_positions = self._get_goat_positions(game_board)
        protected_goats = sum(
            1 for goat in goat_positions
            if all(self.extract_value_from_position(move, game_board) != self.TIGER
                  for move in self.point.get_connections(goat))
        )
        return protected_goats

    def position_in_center(self, game_board=None):
        if game_board is None:
            game_board = self.game_board

        center_positions = ['B2', 'B3', 'B4', 'C2', 'C3', 'C4', 'D2', 'D3', 'D4']
        goat_positions = self._get_goat_positions(game_board)
        return sum(1 for goat in goat_positions if goat in center_positions)

    def avoid_diagonals(self, game_board=None):
        if game_board is None:
            game_board = self.game_board

        diagonal_positions = ['B2', 'B4', 'C1', 'C5', 'D2', 'D4']
        goat_positions = self._get_goat_positions(game_board)
        return sum(1 for goat in goat_positions if goat in diagonal_positions)
